In [3]:
import pandas as pd
import numpy as np

In [4]:
train = pd.read_csv("../MachineHackODIMatchWinner/Train.csv")
test = pd.read_csv("../MachineHackODIMatchWinner/Test.csv")

#sf = pd.read_excel("../MachinehackGlassQualityPrediction/Glass_Quality_Participants_Data/Sample_Submission.xlsx")


In [5]:
nrow = train.shape[0]
train['isTrain'] =1
test['MatchWinner'] =-1
test['isTrain'] =0

data = pd.concat([train,test],axis=0,sort = False).reset_index(drop=True)

In [6]:
train.shape

(2508, 11)

In [7]:
test.shape

(1075, 11)

In [8]:
data['Home2Inning'] = 'OH yeah'
for i in data.index:
    if(data['Team1_Venue'][i] == 'Home' and data['Team1_Innings'][i]=='Second'):
            data['Home2Inning'][i] ='Yes'
    else:
            data['Home2Inning'][i] = 'No'
    

/home/ryan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/ryan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
data['Neutral2Inning'] = 'foo'

for i in data.index:
    if(data['Team1_Venue'][i] == 'Neutral' and data['Team1_Innings'][i]=='Second'):
            data['Neutral2Inning'][i] = 'Yes'
    else:
            data['Neutral2Inning'][i] = 'No'
    

/home/ryan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/home/ryan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [10]:
data['Away2Inning'] = 'foo'
for i in data.index:
    if(data['Team1_Venue'][i] == 'Home' and data['Team1_Innings'][i]=='Second'):
            data['Away2Inning'][i] ='Yes'
    else:
            data['Away2Inning'][i] ='No'
    

/home/ryan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/ryan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
data =  data.drop(['Stadium','Team2_Innings','Team2_Venue'], axis=1)

In [12]:
train = data[data['isTrain']==1]
test = data[data['isTrain']==0]


In [13]:
train = train.drop(['isTrain'],axis=1)
test = test.drop(['MatchWinner', 'isTrain'],axis=1)

In [14]:
def onhotencoder(train,test,cols):
    
    from sklearn.preprocessing import OneHotEncoder
    #object_cols = catVar1(train) #catVar1 gives desired categorical column and not all object columns
    object_cols=cols
    print(object_cols)
    
    OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
    OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(train[object_cols]))
    OH_cols_test = pd.DataFrame(OH_encoder.transform(test[object_cols]))

    # One-hot encoding removed index; put it back
    OH_cols_train.index = train.index
    OH_cols_test.index = test.index

    ##hack for restoring columns names just like get dummies
    column_name = OH_encoder.get_feature_names(object_cols)
    OH_cols_train.columns = column_name
    OH_cols_test.columns = column_name
    

    # Remove desired categorical columns (will replace with one-hot encoding)
    num_train = train.drop(object_cols, axis=1)
    num_test = test.drop(object_cols, axis=1)

    # Add one-hot encoded columns to numerical/remaining features
    OH_train = pd.concat([num_train, OH_cols_train], axis=1)
    OH_test = pd.concat([num_test, OH_cols_test], axis=1)
    
    print(OH_train.shape,OH_test.shape)
    
    
    return OH_train,OH_test

In [15]:
train.columns

Index(['Team1', 'Team2', 'HostCountry', 'Team1_Venue', 'Team1_Innings',
       'MonthOfMatch', 'MatchWinner', 'Home2Inning', 'Neutral2Inning',
       'Away2Inning'],
      dtype='object')

In [16]:
train.columns

Index(['Team1', 'Team2', 'HostCountry', 'Team1_Venue', 'Team1_Innings',
       'MonthOfMatch', 'MatchWinner', 'Home2Inning', 'Neutral2Inning',
       'Away2Inning'],
      dtype='object')

In [17]:
trainlblohe ,testlblohe = onhotencoder(train,test, cols = [ 'Team1_Venue', 'Team1_Innings','MonthOfMatch','Home2Inning','Neutral2Inning','Away2Inning']) 

['Team1_Venue', 'Team1_Innings', 'MonthOfMatch', 'Home2Inning', 'Neutral2Inning', 'Away2Inning']
(2508, 27) (1075, 26)


In [18]:
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from xgboost import XGBClassifier
from ngboost.distns import k_categorical
from ngboost.learners import default_tree_learner
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedShuffleSplit

In [43]:
params = {
        'n_estimators':[50,80,100,125,150,175],
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5],
        'learning_rate': [0.1,0.2,0.3,0.4,0.5]
        }

In [26]:
xgb = XGBClassifier( objective='multi:softprob',
                    silent=True, nthread= -1, random_state=2)


In [27]:
y = trainlblohe.MatchWinner

# To keep things simple, we'll use only numerical predictors
X = trainlblohe.drop(['MatchWinner'], axis=1)



In [28]:
# folds = 5
# param_comb = 20
# sss = StratifiedShuffleSplit(n_splits=10, test_size=0.4, random_state=3)
# sss.get_n_splits(X, y)

# random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='neg_log_loss', n_jobs=-1, cv=sss, verbose=3, random_state=1001 )

# random_search.fit(X, y)


In [29]:
# # print('\n All results:')
# # print(random_search.cv_results_)
# # print('\n Best estimator:')
#  print(random_search.best_estimator_)
# # print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
# # print(random_search.best_score_ * 2 - 1)
# # print('\n Best hyperparameters:')
#  print(random_search.best_params_)

In [30]:
params_grid = {
        'n_estimators':[110,118,125,130,135],
        'min_child_weight': [0.5,1, 1.5,2,3],
        'gamma': [0.3,0.4,0.5,0.7],
        'subsample': [0.9, 1.0,1.1,1.2],
        'colsample_bytree': [0.5,0.6,0.7],
        'learning_rate': [0.2,0.25]
        }

In [31]:
folds = 5
param_comb = 20
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.4, random_state=3)
sss.get_n_splits(X, y)

grid_search = GridSearchCV(xgb, param_grid=params_grid, scoring='neg_log_loss', n_jobs=-1, cv=sss, verbose=3 )

grid_search.fit(X, y)


Fitting 10 folds for each of 2400 candidates, totalling 24000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   13.7s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:   42.7s
[Parallel(n_jobs=-1)]: Done 288 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 541 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 861 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 1216 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-1)]: Done 1648 tasks      | elapsed:  9.6min
[Parallel(n_jobs=-1)]: Done 2192 tasks      | elapsed: 12.8min
[Parallel(n_jobs=-1)]: Done 2785 tasks      | elapsed: 16.1min
[Parallel(n_jobs=-1)]: Done 3509 tasks      | elapsed: 20.0min
[Parallel(n_jobs=-1)]: Done 4270 tasks      | elapsed: 24.0min
[Parallel(n_jobs=-1)]: Done 5144 tasks      | elapsed: 28.8min
[Parallel(n_jobs=-1)]: Done 6048 tasks      | elapsed: 33.9min
[Parallel(n_jobs=-1)]: Done 7008 tasks      | elapsed: 39.3min
[Parallel(n_jobs=-1)]: Done 8084 tasks      | e

KeyboardInterrupt: 

In [35]:
grid_search.get_params

<bound method BaseEstimator.get_params of GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=3, test_size=0.4,
            train_size=None),
             error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=-1...
                                     scale_pos_weight=1, seed=None, silent=True,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=-1,
             param_grid={'colsample_bytree': [0.5, 0.6, 0.7],
                         'gamma': [0.3, 0.4, 0.5, 0.7],
                     

In [83]:
from ngboost import NGBClassifier
from ngboost.distns import k_categorical
from ngboost.learners import default_tree_learner
clf2 = NGBClassifier(random_state=2,Dist=k_categorical(16))
clf2.fit(X,y)

[iter 0] loss=2.3886 val_loss=0.0000 scale=0.5000 norm=9.4365
[iter 100] loss=1.4553 val_loss=0.0000 scale=1.0000 norm=5.6311
[iter 200] loss=0.9715 val_loss=0.0000 scale=1.0000 norm=3.6072
[iter 300] loss=0.8678 val_loss=0.0000 scale=1.0000 norm=3.3070
[iter 400] loss=0.8126 val_loss=0.0000 scale=1.0000 norm=3.1493


NGBClassifier(Base=DecisionTreeRegressor(ccp_alpha=0.0,
                                         criterion='friedman_mse', max_depth=3,
                                         max_features=None, max_leaf_nodes=None,
                                         min_impurity_decrease=0.0,
                                         min_impurity_split=None,
                                         min_samples_leaf=1,
                                         min_samples_split=2,
                                         min_weight_fraction_leaf=0.0,
                                         presort='deprecated',
                                         random_state=None, splitter='best'),
              Dist=<class 'ngboost.distns.categorical.k_categorical.<locals>.Categorical'>,
              Score=<class 'ngboost.scores.LogScore'>, learning_rate=0.01,
              minibatch_frac=1.0, n_estimators=500, natural_gradient=True,
              random_state=RandomState(MT19937) at 0x7F4CB371EDB0, tol=0

In [118]:
from ngboost.distns import k_categorical
from ngboost import NGBClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier


b1 = DecisionTreeRegressor(criterion="friedman_mse", max_depth=2)
b2 = DecisionTreeRegressor(criterion="friedman_mse", max_depth=4)
b3 = Ridge(alpha=0.0)
b4 = XGBClassifier(random_state=2)
b5 = LGBMClassifier(random_state=2)

param_grid = {
        "n_estimators": [20, 50],
        "minibatch_frac": [1.0, 0.5],
        "Base": [b1,b3],
    }

ngb = NGBClassifier(natural_gradient=True, verbose=False, Dist=k_categorical(16))

grid_search = GridSearchCV(ngb, param_grid=param_grid, cv=5)
grid_search.fit(X, y)
print(grid_search.best_params_)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
TypeError: '(array([ 453,  154, 1701, ...,  816,  893,  451]), slice(None, None, None))' is an invalid key

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
TypeError: '(array([1500,  154,  989, ..., 1903,  816,  893]), slice(None, None, None))' is an invalid key

  FitFailedWarning)
/home/ryan/.local/lib/python3.6/site-packages/ngboost/distns/categorical.py:9: RuntimeWarning: divide by zero encountered in log
  return -np.log(self.probs[Y, range(len(Y))])
/home/ryan/.local/lib/python3.6/site-packages/ngboost/distns/categorical.py:9: RuntimeWarning: divide by zero encountered in log
  return -np.

{'Base': Ridge(alpha=0.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001), 'minibatch_frac': 1.0, 'n_estimators': 20}


In [ ]:
def score_dataset(X_train, X_valid, y_train, y_valid):
    
    from xgboost import XGBClassifier
    from ngboost.distns import k_categorical
    from ngboost.learners import default_tree_learner
    from sklearn.metrics import log_loss
    
    from sklearn.metrics import classification_report
    from sklearn.tree import DecisionTreeClassifier
    from ngboost import NGBClassifier
    from sklearn.ensemble import BaggingClassifier
    from lightgbm import LGBMClassifier
    from sklearn.calibration import CalibratedClassifierCV
    from sklearn.model_selection import StratifiedKFold, KFold,StratifiedShuffleSplit
    from scipy.sparse import csr_matrix, hstack




         
    
    clf2 = XGBClassifier(random_state=2)
    
    #clf2 = NGBClassifier(random_state=2,Dist=k_categorical(16))
    
    clf2.fit(X_train,y_train)
    
    sss = StratifiedShuffleSplit(n_splits=10, test_size=0.4, random_state=3)
    sss.get_n_splits(X_train, y_train)
    
    clf_sigmoid = CalibratedClassifierCV(clf2, cv=sss, method='isotonic')
    clf_sigmoid.fit(X_train, y_train)
    preds = clf_sigmoid.predict_proba(X_valid)
    preds1 = clf_sigmoid.predict(X_valid)

#     preds1 = clf2.predict(X_valid)
#     preds = clf2.predict_proba(X_valid)
    target_names = ['class 0', 'class 1', 'class 2','class 3', 'class 4', 'class 5', 'class 6',
                    'class 7', 'class 8', 'class 9','class 10', 'class 11', 'class 12', 'class 13',
                    'class 14', 'class 15']
    #print(preds)
    print(classification_report(y_valid, preds1, target_names=target_names,labels= [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]))
    return log_loss(y_valid,preds) 

In [ ]:
###### from sklearn.model_selection import train_test_split
y = trainlblohe.MatchWinner

# To keep things simple, we'll use only numerical predictors
X = trainlblohe.drop(['MatchWinner'], axis=1)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.57, test_size=0.43,random_state=0)